  ---
 # Структура проекта
 ### 1.Выгрузка данных
 -  Ознакомление и предобработка данных из таблицы с категориями фильмов
 -  Выгрузка субтитров и сопоставление с названиями фильмов
 -  Очистка от дубликатов
 -  Создание сводной таблицы с данными
 -  Выгрузка словаря 
 ### 2.Фильтрация слов и разбиение на ключевые блоки
 -  Лемматизация и добавление новых фитч на основе словаря оксфорда
 -  Исключение стоп сов из данных
 ### 3.Препроцессинг
 -  Разбиение данных на выборки
 -  Ознакомление с дисбалансом
 ### 4. Обучение моделей
 ### 5. Вывод  
 --- 
 

### 1.Выгрузка данных

In [1]:
%pip  install openpyxl
%pip install pysrt
%pip install spacy
%pip install sklearn
%pip install scikit-learn
%pip install nltk
%pip install reportlab



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [449]:
import sys
import io
!{sys.executable} -m pip install pandas pysrt nltk pyprind joblib
import pandas as pd
import numpy as np
import glob
import os
import string
import spacy 
import pysrt
import re
from time import time
from joblib import dump, load
import pyprind
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import PyPDF2
from reportlab.pdfgen import canvas
from openpyxl import Workbook
import matplotlib.pyplot as plt



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [450]:
from sklearn.datasets import load_files 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.extmath import density
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.metrics import classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from PyPDF2 import PdfReader

-  Ознакомление и предобработка данных из таблицы с категориями фильмов

In [451]:
dataeng='movies_labels.xlsx'
data = pd.read_excel(dataeng)
data.head()

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [452]:
data[data.duplicated(keep=False)]

,id,Movie,Level


Проверим уникальность значений и проведем перекодировку признаков, назначив индексы

In [453]:
data['Level'].unique()

array(['B1', 'B2', 'A2/A2+', 'C1', 'B1, B2', 'A2/A2+, B1', 'A2'],
      dtype=object)

- Заменим имеющиеся категории на соответствующие, объеденив некоторые.

In [454]:
new_level = {
  'B1': 2,
  'B2': 3, 
  'A2/A2+': 1,
  'C1': 4,
  'B1, B2': 3, 
  'A2/A2+, B1': 2, 
  'A2': 1
  }

In [455]:
data = data.replace(new_level)
data.head()

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),2
1,1,10_things_I_hate_about_you(1999),2
2,2,A_knights_tale(2001),3
3,3,A_star_is_born(2018),3
4,4,Aladdin(1992),1


In [456]:
data[data.duplicated(keep=False)]

,id,Movie,Level


Дубликатов не обнаружено



 -  Выгрузка словаря 
 

 

Создадим файл с колонками соответствующими уровню английского

Прочитаем Pdf файл со словарем и трансформируем в xl

In [458]:
# Открываем PDF файл
pdf_file = open('\\Users\Asus\Desktop\All_project\Category_of_english_Level\Oxford_CEFR_level\The_Oxford_5000_by_CEFR_level.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Создаем новый файл Excel
wb = Workbook()
ws = wb.active

# Извлекаем текст из каждой страницы PDF файла
for page in pdf_reader.pages:
    text = page.extract_text()
    
    # Записываем текст в ячейку Excel
    ws.append([text])

# Сохраняем файл Excel
wb.save('output.xlsx')

# Закрываем PDF файл
pdf_file.close()

Осущетвили ручной перенос слов, разделяя их по категориям

Взглянем что получилось

In [459]:
vocab = pd.read_excel('\\Users\Asus\Desktop\All_project\Category_of_english_Level\English_level_env\words_category.xlsx')
vocab.head()

,A2,B1,B2,C1
0,ability,absolutely academic,abandon,abolish
1,able,access,absolute,abortion
2,abroad,accommodation,academic,absence
3,accept,account,acceptable,absent
4,accident,achievement,accompany,absurd


Данные требуют чистки, поэтому удалим лишние символы и слова

In [460]:
vocab = vocab.astype(str)
# Создадим функцию для очистки элементов колонки
def clean_column_value(value):
    # Удалите лишние символы и слова с помощью регулярных выражений
    cleaned_value = re.sub(r'\W+', ' ', value)  # удаление не алфавитных символов
    cleaned_value = re.sub(r'\b\w{1,2}\b', '', cleaned_value)  # удаление слов меньше 3 символов   
    stop_words_vocab = ['adv','pron', 'det', 'adj', 'prep', 'modal', 'exclam']

    # Формируем шаблон для всех стоп-слов в виде '|word1|word2|word3|...'
    pattern = '|'.join(stop_words_vocab)

    # Заменяем все совпадения со стоп-словами на пустую строку
    cleaned_value = re.sub(pattern, '', cleaned_value)

    return cleaned_value 
    # Применим функцию ко всем колонкам датасета #for column in vocab: 

vocab['A2'] = vocab['A2'].apply(clean_column_value)
vocab['B1'] = vocab['B1'].apply(clean_column_value)
vocab['B2'] = vocab['B2'].apply(clean_column_value)
vocab['C1'] = vocab['C1'].apply(clean_column_value)
    # Выведите очищенную колонку на экран


In [461]:
print(vocab.head())

          A2                    B1           B2         C1
0   ability   absolutely academic      abandon    abolish 
1      able                access     absolute   abortion 
2    abroad         accommodation     academic    absence 
3    accept               account   acceptable     absent 
4  accident           achievement    accompany     absurd 


In [462]:
vocab = vocab.fillna('')

Создадим дополнительные словари под каждый уровень

In [464]:
dict_A2 = vocab['A2'][0:860].to_dict()
dict_B1 = vocab['B1'][0:797].to_dict()
dict_B2 = vocab['B2'][0:].to_dict()
dict_C1 = vocab['C1'][0:1117].to_dict()
dict_A2 = list(dict_A2.values())
dict_B1 = list(dict_B1.values())
dict_B2 = list(dict_B2.values())
dict_C1 = list(dict_C1.values()) #all_dict
all_dict = [dict_A2, dict_B1, dict_B2, dict_C1]
all_dict[3]

['abolish ',
 'abortion ',
 'absence ',
 'absent ',
 'absurd ',
 'abundance ',
 'abuse ',
 'academy ',
 'accelerate ',
 'acceptance ',
 'accessible ',
 'accomplishment ',
 'accordance ',
 'accordingly ',
 'accountability ',
 'accountable ',
 'accumulate ',
 'accumulation ',
 'accusation ',
 'accused ',
 'acid ',
 'acquisition ',
 'acre ',
 'activation activist ',
 'acute ',
 'adaptation ',
 'adhere ',
 'acent ',
 'ustment ',
 'administer ',
 'administrative ',
 'administrator ',
 'admission ',
 'adolescent ',
 'adoption ',
 'erse ',
 'ocate ',
 'aesthetic ',
 'affection ',
 'aftermath ',
 'aggression ',
 'agricultural ',
 'aide ',
 'albeit conj ',
 'alert ',
 'alien ',
 'align ',
 'alignment ',
 'alike ',
 'allegation ',
 'allege ',
 'allegedly ',
 'alliance ',
 'allocate ',
 'allocation ',
 'allowance ',
 'ally ',
 'aluminium ',
 'amateur ',
 'ambassador ',
 'amend ',
 'amendment ',
 'amid ',
 'analogy ',
 'anchor ',
 'angel ',
 'anonymous ',
 'apparatus ',
 'appealing ',
 'appetite '

-  Выгрузка субтитров и сопоставление с названиями фильмов

In [465]:
# загрузим в список имена файлов из папки с субтитрами
films_call = os.listdir(path='.\Subtitles_all\Subtitles')
# определим количество названий фильмов
print(f'Количество названий фильмов в папке с субтитрами: {len(films_call)}')

Количество названий фильмов в папке с субтитрами: 279


In [466]:
# проверим для скольких фильмов, имеющих метку из таблицы, предоставлены субтитры 
films_filtr = set(films_call) & set(data['Movie'] + '.srt')
print(f'Количество фильмов, имеющих метку и субтитры: {len(films_filtr)}')

Количество фильмов, имеющих метку и субтитры: 229


Подготовим фильтры для субтитров

In [467]:
# Введем переменные для очистки текста
HTML = r'<.*?>' # html тэги меняем на пробел
TAG = r'{.*?}' # тэги меняем на пробел
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]' # комменты в скобках меняем на пробел
LETTERS = r'[^a-zA-Z\.,!? ]' # все что не буквы меняем на пробел 
SPACES = r'([ ])\1+' # повторяющиеся пробелы меняем на один пробел
SYMB = r'[^\w\s]' # знаки припинания
LONG = r'\b\w{1,2}\b'# удаление слов менее трех символов
DOTS = r'[\.]+' # многоточие меняем на точку
# SYMB = r"[^\w\d'\s]" # знаки препинания кроме апострофа


In [468]:
# напишем функцию для очистки субтитров
def clean_subs(subs):
    subs = subs[1:] # удаляем первый рекламный субтитр
    txt = re.sub(HTML, ' ', subs.text) # html тэги меняем на пробел
    txt = re.sub(COMMENTS, ' ', txt) # комменты в скобках меняем на пробел
    txt = re.sub(LETTERS, ' ', txt) # все что не буквы меняем на пробел
    txt = re.sub(DOTS, r'.', txt) # многоточие меняем на точку
    txt = re.sub(SPACES, r'\1', txt) # повторяющиеся пробелы меняем на один пробел
    #txt = re.sub(LONG, r'\b\w{1,2}\b', '', txt) # удаление менее трех символов
    txt = re.sub(SYMB, '', txt) # знаки препинания кроме апострофа на пустую строку
    txt = re.sub('www', '', txt) # кое-где остаётся www, то же меняем на пустую строку
    txt = txt.lstrip() # обрезка пробелов слева
    txt = txt.encode('ascii', 'ignore').decode() # удаляем все что не ascii символы   
    txt = txt.lower() # текст в нижний регистр
    return txt

- Подготовим функцию для лемматизации и сортировки используя для каждокй категории свой словарь, подгодтовленный ранее

In [432]:
# функция возвращающая количество уникальных лемм
# в тексте субтитров по каждому уровню
def lemma_count(lemmas, oxf, cat):
    func_dict = {"A2": 1, 
                 "B1": 2, 
                 "B2": 3, 
                 "C1": 4}
    level = func_dict[cat]
    oxf_word_list = oxf[level]#.split()
    words = [lemma for lemma in lemmas if lemma in oxf_word_list]

    return len(set(words))
# загрузим для каждого фильма субтитры с использованием библиотеки pysrt

for film in films_filtr:
    #try: 
    subs = pysrt.open(f'.\Subtitles_all\Subtitles\{film}', encoding='iso-8859-1')
    #except:
    #    subs = pysrt.open(f'.\Subtitles_all\Subtitles{film}', encoding='iso-8859-1')
    # вызов функии для очистки текста

    cln_subs = clean_subs(subs)
    data.loc[data['Movie'] == film[:-4], 'subs'] = cln_subs
    # используем библиотеку spacy для лемматизации 
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(cln_subs)
    lemma_list = [token.lemma_ for token in doc]
    for lvl in ['A2', 'B1', 'B2', 'C1']:
        data.loc[data['Movie'] == film[:-4], lvl+'_lemma_cnt'] = lemma_count(lemma_list, dict_A2, lvl)
    dffr_A2 = pd.DataFrame(data, columns=['id', 'Level', 'Movie','A2_lemma_cnt', 'B1_lemma_cnt', 'B2_lemma_cnt', 'C1_lemma_cnt' , 'subs'])

In [469]:
dffr_A2

,id,Level,Movie,A2_lemma_cnt,B1_lemma_cnt,B2_lemma_cnt,C1_lemma_cnt,subs
0,0,2,10_Cloverfield_lane(2016),3.0,6.0,4.0,6.0,ben on phone michelle please don t hang up jus...
1,1,2,10_things_I_hate_about_you(1999),5.0,5.0,5.0,7.0,i ll be right with you so cameron here you go ...
2,2,3,A_knights_tale(2001),4.0,4.0,3.0,6.0,should we help him he s due in the lists in tw...
3,3,3,A_star_is_born(2018),3.0,4.0,3.0,7.0,get to it black eyes open wide it s time to te...
4,4,1,Aladdin(1992),5.0,6.0,4.0,6.0,where the caravan camels roam where it s flat ...
...,...,...,...,...,...,...,...,...
236,236,4,Matilda(2022),4.0,4.0,4.0,4.0,NaN
237,237,2,Bullet train,4.0,4.0,4.0,4.0,NaN
238,238,3,Thor: love and thunder,4.0,4.0,4.0,4.0,NaN
239,239,3,Lightyear,4.0,4.0,4.0,4.0,NaN


- Для B1

In [470]:
for film in films_filtr:
    subs = pysrt.open(f'.\Subtitles_all\Subtitles\{film}', encoding='iso-8859-1')
    cln_subs = clean_subs(subs)
    data.loc[data['Movie'] == film[:-4], 'subs'] = cln_subs
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(cln_subs)
    lemma_list = [token.lemma_ for token in doc]
    for lvl in ['A2', 'B1', 'B2', 'C1']:
        data.loc[data['Movie'] == film[:-4], lvl+'_lemma_cnt'] = lemma_count(lemma_list, dict_B1, lvl)
    dffr_B1 = pd.DataFrame(data, columns=['id', 'Movie', 'A2_lemma_cnt', 'B1_lemma_cnt', 'B2_lemma_cnt', 'C1_lemma_cnt', 'subs'])

In [473]:
dffr_B1

,id,Movie,A2_lemma_cnt,B1_lemma_cnt,B2_lemma_cnt,C1_lemma_cnt,subs
0,0,10_Cloverfield_lane(2016),3.000000,10.000000,5.000000,6.000000,ben on phone michelle please don t hang up jus...
1,1,10_things_I_hate_about_you(1999),4.000000,8.000000,5.000000,9.000000,i ll be right with you so cameron here you go ...
2,2,A_knights_tale(2001),3.000000,7.000000,4.000000,5.000000,should we help him he s due in the lists in tw...
3,3,A_star_is_born(2018),3.000000,8.000000,3.000000,8.000000,get to it black eyes open wide it s time to te...
4,4,Aladdin(1992),4.000000,10.000000,6.000000,9.000000,where the caravan camels roam where it s flat ...
...,...,...,...,...,...,...,...
236,236,Matilda(2022),5.873391,5.873391,5.873391,5.873391,NaN
237,237,Bullet train,5.873391,5.873391,5.873391,5.873391,NaN
238,238,Thor: love and thunder,5.873391,5.873391,5.873391,5.873391,NaN
239,239,Lightyear,5.873391,5.873391,5.873391,5.873391,NaN


 - Для B2

In [474]:
for film in films_filtr:
    subs = pysrt.open(f'.\Subtitles_all\Subtitles\{film}', encoding='iso-8859-1')
    cln_subs = clean_subs(subs)
    data.loc[data['Movie'] == film[:-4], 'subs'] = cln_subs
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(cln_subs)
    lemma_list = [token.lemma_ for token in doc]
    for lvl in ['A2', 'B1', 'B2', 'C1']:
        data.loc[data['Movie'] == film[:-4], lvl+'_lemma_cnt'] = lemma_count(lemma_list, dict_B2, lvl)
        dffr_B2 = pd.DataFrame(data, columns=['id', 'Movie', 'A2_lemma_cnt', 'B1_lemma_cnt', 'B2_lemma_cnt', 'C1_lemma_cnt', 'subs'])

In [475]:
dffr_B2

,id,Movie,A2_lemma_cnt,B1_lemma_cnt,B2_lemma_cnt,C1_lemma_cnt,subs
0,0,10_Cloverfield_lane(2016),6.000000,5.000000,7.000000,7.000000,ben on phone michelle please don t hang up jus...
1,1,10_things_I_hate_about_you(1999),8.000000,9.000000,9.000000,6.000000,i ll be right with you so cameron here you go ...
2,2,A_knights_tale(2001),6.000000,5.000000,5.000000,6.000000,should we help him he s due in the lists in tw...
3,3,A_star_is_born(2018),5.000000,8.000000,5.000000,6.000000,get to it black eyes open wide it s time to te...
4,4,Aladdin(1992),9.000000,7.000000,8.000000,7.000000,where the caravan camels roam where it s flat ...
...,...,...,...,...,...,...,...
236,236,Matilda(2022),5.851931,5.851931,5.851931,5.851931,NaN
237,237,Bullet train,5.851931,5.851931,5.851931,5.851931,NaN
238,238,Thor: love and thunder,5.851931,5.851931,5.851931,5.851931,NaN
239,239,Lightyear,5.851931,5.851931,5.851931,5.851931,NaN


- Для C1

In [476]:
for film in films_filtr:
    subs = pysrt.open(f'.\Subtitles_all\Subtitles\{film}', encoding='iso-8859-1')
    cln_subs = clean_subs(subs)
    data.loc[data['Movie'] == film[:-4], 'subs'] = cln_subs
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(cln_subs)
    lemma_list = [token.lemma_ for token in doc]
    for lvl in ['A2', 'B1', 'B2', 'C1']:
        data.loc[data['Movie'] == film[:-4], lvl+'_lemma_cnt'] = lemma_count(lemma_list, dict_C1, lvl)
        dffr_C1 = pd.DataFrame(data, columns=['id', 'Movie', 'A2_lemma_cnt', 'B1_lemma_cnt', 'B2_lemma_cnt', 'C1_lemma_cnt', 'subs'])

In [478]:
dffr_C1

,id,Movie,A2_lemma_cnt,B1_lemma_cnt,B2_lemma_cnt,C1_lemma_cnt,subs
0,0,10_Cloverfield_lane(2016),8.000000,3.000000,4.000000,5.000000,ben on phone michelle please don t hang up jus...
1,1,10_things_I_hate_about_you(1999),6.000000,4.000000,5.000000,5.000000,i ll be right with you so cameron here you go ...
2,2,A_knights_tale(2001),5.000000,4.000000,4.000000,4.000000,should we help him he s due in the lists in tw...
3,3,A_star_is_born(2018),6.000000,3.000000,4.000000,4.000000,get to it black eyes open wide it s time to te...
4,4,Aladdin(1992),7.000000,6.000000,6.000000,5.000000,where the caravan camels roam where it s flat ...
...,...,...,...,...,...,...,...
236,236,Matilda(2022),4.639485,4.639485,4.639485,4.639485,NaN
237,237,Bullet train,4.639485,4.639485,4.639485,4.639485,NaN
238,238,Thor: love and thunder,4.639485,4.639485,4.639485,4.639485,NaN
239,239,Lightyear,4.639485,4.639485,4.639485,4.639485,NaN


- Проведем сбор датафрейма на основе лемматизации

- Соберем новый фрейм и переименуем столбцы

In [492]:
new_featch = pd.concat([dffr_A2.iloc[:, -1: ], dffr_A2.iloc[:, 1:7],  dffr_B1.iloc[:, 2:6], dffr_B2.iloc[:, 2:6], dffr_C1.iloc[:, 2:6]], ignore_index=True, axis=1)

In [493]:
new_featch.rename(columns={new_featch.columns[0] : 'subs', new_featch.columns[1] : 'Level', new_featch.columns[2] : 'Movie', new_featch.columns[3] : 'A2_A2', new_featch.columns[4] : 'B1_A2', new_featch.columns[5] : 'B2_A2', new_featch.columns[6] : 'C1_A2', new_featch.columns[7] : 'A2_B1', new_featch.columns[8] : 'B1_B1', new_featch.columns[9] : 'B2_B1', new_featch.columns[10] : 'C1_B1', new_featch.columns[11] : 'A2_B2', new_featch.columns[12] : 'B1_B2', new_featch.columns[13] : 'B2_B2', new_featch.columns[14] : 'C1_B2', new_featch.columns[15] : 'A2_C1', new_featch.columns[16] : 'B1_C1', new_featch.columns[17] : 'B2_C1',new_featch.columns[18] : 'C1_C1', new_featch.columns[0] : 'subs', new_featch.columns[1] : 'Level', new_featch.columns[2] : 'Movie', new_featch.columns[3] : 'A2_A2', new_featch.columns[4] : 'B1_A2', new_featch.columns[5] : 'B2_A2', new_featch.columns[6] : 'C1_A2', new_featch.columns[7] : 'A2_B1', new_featch.columns[8] : 'B1_B1', new_featch.columns[9] : 'B2_B1', new_featch.columns[10] : 'C1_B1', new_featch.columns[11] : 'A2_B2', new_featch.columns[12] : 'B1_B2', new_featch.columns[13] : 'B2_B2', new_featch.columns[14] : 'C1_B2', new_featch.columns[15] : 'A2_C1', new_featch.columns[16] : 'B1_C1', new_featch.columns[17] : 'B2_C1',new_featch.columns[18] : 'C1_C1'}, inplace=True)

In [494]:
new_featch.head()

,subs,Level,Movie,A2_A2,B1_A2,B2_A2,C1_A2,A2_B1,B1_B1,B2_B1,C1_B1,A2_B2,B1_B2,B2_B2,C1_B2,A2_C1,B1_C1,B2_C1,C1_C1
0,ben on phone michelle please don t hang up jus...,2,10_Cloverfield_lane(2016),3.0,6.0,4.0,6.0,3.0,10.0,5.0,6.0,6.0,5.0,7.0,7.0,8.0,3.0,4.0,5.0
1,i ll be right with you so cameron here you go ...,2,10_things_I_hate_about_you(1999),5.0,5.0,5.0,7.0,4.0,8.0,5.0,9.0,8.0,9.0,9.0,6.0,6.0,4.0,5.0,5.0
2,should we help him he s due in the lists in tw...,3,A_knights_tale(2001),4.0,4.0,3.0,6.0,3.0,7.0,4.0,5.0,6.0,5.0,5.0,6.0,5.0,4.0,4.0,4.0
3,get to it black eyes open wide it s time to te...,3,A_star_is_born(2018),3.0,4.0,3.0,7.0,3.0,8.0,3.0,8.0,5.0,8.0,5.0,6.0,6.0,3.0,4.0,4.0
4,where the caravan camels roam where it s flat ...,1,Aladdin(1992),5.0,6.0,4.0,6.0,4.0,10.0,6.0,9.0,9.0,7.0,8.0,7.0,7.0,6.0,6.0,5.0


- Обработаем пропуски

In [501]:
# найдем пропуски в данных после загрузки
new_featch[new_featch['subs'].isna()].sort_values('Movie')

,subs,Level,Movie,A2_A2,B1_A2,B2_A2,C1_A2,A2_B1,B1_B1,B2_B1,C1_B1,A2_B2,B1_B2,B2_B2,C1_B2,A2_C1,B1_C1,B2_C1,C1_C1
237,NaN,2,Bullet train,4.0,4.0,4.0,4.0,5.873391,5.873391,5.873391,5.873391,5.851931,5.851931,5.851931,5.851931,4.639485,4.639485,4.639485,4.639485
235,NaN,3,Glass Onion,4.0,4.0,4.0,4.0,5.873391,5.873391,5.873391,5.873391,5.851931,5.851931,5.851931,5.851931,4.639485,4.639485,4.639485,4.639485
239,NaN,3,Lightyear,4.0,4.0,4.0,4.0,5.873391,5.873391,5.873391,5.873391,5.851931,5.851931,5.851931,5.851931,4.639485,4.639485,4.639485,4.639485
236,NaN,4,Matilda(2022),4.0,4.0,4.0,4.0,5.873391,5.873391,5.873391,5.873391,5.851931,5.851931,5.851931,5.851931,4.639485,4.639485,4.639485,4.639485
240,NaN,2,The Grinch,4.0,4.0,4.0,4.0,5.873391,5.873391,5.873391,5.873391,5.851931,5.851931,5.851931,5.851931,4.639485,4.639485,4.639485,4.639485
81,NaN,3,The Secret Life of Pets.en,4.0,4.0,4.0,4.0,5.873391,5.873391,5.873391,5.873391,5.851931,5.851931,5.851931,5.851931,4.639485,4.639485,4.639485,4.639485
238,NaN,3,Thor: love and thunder,4.0,4.0,4.0,4.0,5.873391,5.873391,5.873391,5.873391,5.851931,5.851931,5.851931,5.851931,4.639485,4.639485,4.639485,4.639485
106,NaN,1,Up (2009),4.0,4.0,4.0,4.0,5.873391,5.873391,5.873391,5.873391,5.851931,5.851931,5.851931,5.851931,4.639485,4.639485,4.639485,4.639485


 - Посчитаем для каждого уровня с вероятностями среднее значение чтобы каждый уровень заполнить своим 

In [502]:
# выдедем отсоритрованный список фильмов  для которых отсутствую субтитры в папке `'/Datasets/Subtitles'`
sorted(set(new_featch['Movie'] + '.srt') - set(films_call))

['Bullet train.srt',
 'Glass Onion.srt',
 'Lightyear.srt',
 'Matilda(2022).srt',
 'The Grinch.srt',
 'The Secret Life of Pets.en.srt',
 'Thor: love and thunder.srt',
 'Up (2009).srt']

In [503]:
# удалим фильмы с пропущенными субтитрами
new_featch.dropna(inplace=True)
# проверим удаление
new_featch[new_featch['subs'].isna()]

,subs,Level,Movie,A2_A2,B1_A2,B2_A2,C1_A2,A2_B1,B1_B1,B2_B1,C1_B1,A2_B2,B1_B2,B2_B2,C1_B2,A2_C1,B1_C1,B2_C1,C1_C1


- Оценим дисбаланс классов

In [504]:
new_featch['Level'].value_counts()

Level
3    105
2     58
4     39
1     31
Name: count, dtype: int64

Присутсвует дисбаланс в сторону 3 класса. Проводить манипуляции не целесообразно, так как наличие текста подразумевает значительное влиеяние на результат в случае добавления данных.

### 3.Препроцессинг

- Векторизация текстовых признаков признаков

In [515]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(new_featch['subs'])

tfidf_df = pd.DataFrame(X.toarray(), index=new_featch['Movie'], columns = vectorizer.get_feature_names_out())
all_data = new_featch.merge(tfidf_df, on='Movie', how='left')

In [516]:
all_data.shape

(241, 31452)

In [572]:
all_data

,subs_x,Level,Movie,A2_A2,B1_A2,B2_A2,C1_A2,A2_B1,B1_B1,B2_B1,...,zorbing,zounds,zpd,zsa,zsaz,zuckerberg,zuk,zulu,zurg,zuzu
0,ben on phone michelle please don t hang up jus...,2,10_Cloverfield_lane(2016),3.0,6.0,4.0,6.0,3.0,10.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,i ll be right with you so cameron here you go ...,2,10_things_I_hate_about_you(1999),5.0,5.0,5.0,7.0,4.0,8.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,should we help him he s due in the lists in tw...,3,A_knights_tale(2001),4.0,4.0,3.0,6.0,3.0,7.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,get to it black eyes open wide it s time to te...,3,A_star_is_born(2018),3.0,4.0,3.0,7.0,3.0,8.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,where the caravan camels roam where it s flat ...,1,Aladdin(1992),5.0,6.0,4.0,6.0,4.0,10.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,i assume my deal with edward is dead as long a...,4,Suits.S03E06.720p.HDTV.x264-mSD,3.0,5.0,5.0,5.0,5.0,11.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
237,it s going up on the wall tomorrow and this is...,4,Suits.S03E07.HDTV.x264-mSD,3.0,5.0,4.0,5.0,4.0,8.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
238,darby backs me for managing partner i don t wa...,4,Suits.S03E08.480p.HDTV.x264-mSD,2.0,3.0,4.0,4.0,3.0,9.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
239,i m bonding with your father here speaking of ...,4,Suits.S03E09.480p.HDTV.x264-mSD,2.0,3.0,4.0,4.0,3.0,7.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- Разделим данные на выборки

In [506]:
RANDOM_STATE = 42

In [533]:
# разобъем данные на валидационную и тестовую выборки
data_train, data_test = train_test_split(all_data, random_state=RANDOM_STATE, test_size=.2)
# определим независимые признаки и зависимую целевую метку
X_train = data_train.drop(['Level', 'Movie', 'subs_x'], axis=1)
y_train = data_train['Level']
X_test = data_test.drop(['Level', 'Movie', 'subs_x'], axis=1)
y_test = data_test['Level']
# определим размеры обучающей и тестовой выборки
print(f'Размер обучающей выборки: {X_train.shape, y_train.shape}')
print(f'Размер тестовой выборки: {X_test.shape, y_test.shape}')

Размер обучающей выборки: ((192, 31449), (192,))
Размер тестовой выборки: ((49, 31449), (49,))


### 4. Обучим модели

In [539]:
%pip install catboost
from catboost import CatBoostClassifier as cb

In [545]:
model = cb(
    random_seed=42,
    iterations=50,
    learning_rate=0.05
)
       
model.fit(
    X_train, y_train,
    verbose=50
)
#pred = model.predict(X_test)
#print(classification_report(y_test, pred))

0:	learn: 1.3628477	total: 1.63s	remaining: 1m 19s
49:	learn: 0.7761546	total: 32.3s	remaining: 0us


In [546]:
pred = model.predict(X_test)

In [547]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.41      0.64      0.50        11
           3       0.63      0.81      0.71        21
           4       1.00      0.45      0.62        11

    accuracy                           0.59        49
   macro avg       0.51      0.48      0.46        49
weighted avg       0.59      0.59      0.56        49



C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: U

# 5.Вывод

 ---

- Итак имеем точность предсказаний 0.59. Другими словами уже не ноль(0.5), но и не максимальные горизонты возможного. Можно утверждать, что модель работает, но  совершенству нет предела и возможны доработки в разных направлениях. 
- Для выпуска модели в продакшн,  наряду с интерфейсом, конечно же потребуется расширение исходной базы данных как фильмов, так и дополнительных списков слов, которые обогатят и сделают более точной модель.
- Так как уровни языка это очень субъективная часть жизни и каждый уровень более точен с более широким контекстом. Кроме экспериментов с моделями и гиперпараметрами, входящие данные остаются во главе угла. 
-  В инжиниринг слудует довносить и структурировать данные с различных сфер. И если говорить о идеальной модели, то базовая версия состоит из условных 25% основной информации, то оставшаяся, не осознанная и по каким то причинам не внесенная, растягивается длинным хвостом не упорядоченная, полезная информация, которая в объеме значительно привышает текущий объем.  